In [25]:
import pymongo as py
from collections import defaultdict
import json
MONGO_URI = "XXXX"

print(MONGO_URI)

mongodb://cosmo-feynman:4rDaE4LwLhwtwKCRIID7BV42Dpz9JOcj3H8kTDUqDmDbkmJiW4HqcGmktibeGWdAkFE5ZE8bQPvR1EncGLSiGA==@cosmo-feynman.documents.azure.com:10255/?ssl=true&replicaSet=globaldb


In [33]:
import spacy
spacy.load('en')

In [18]:
import re
accounts_file = open('C:/Users/alist/Test Texts/test01.txt', encoding='utf-8')
accounts_txt = accounts_file.read()
accounts_txt = re.sub(r'\n', ' ', accounts_txt)


In [55]:
def text_cleaner(text):
    
    """Lemmatizes tokens and removes stop words, punctuation, and numbers. Identifies named entities. 
    Returns a dictionary of lists keyed on tokens and different types of named entities keyed in the form
    'Named Entity - ORG', see spacy docs for the list of entity types  
    
    Arguments:
    text - raw text to tokenize may need to run re.sub(r'\n', ' ', text) to clean up
    
    """
    
    #load the text into spacy to tag
    nlp = spacy.load('en', tagger=False, parser=False, matcher=False)
    doc = nlp(text)
    
    #TODO - break up the text by page or paragraph using repeat newlines?
    
    #setup output as a default dictionary of lists
    out_dict = defaultdict(list)
    
    #populate the tokes slot removing spaces, numbers and punctuation etc.
    out_dict['tokens'] = [ t.lemma_ for t in doc if not (t.is_stop or t.pos_ in ["PUNCT", "SPACE", "NUM"] or t.text == "\ufeff") ]

    #populate the named entity slots first removing any entities that are just whitespace
    doc.ents = [e for e in doc.ents if not e.text.isspace()]
    for ent in doc.ents:
        out_dict['Named Entity - '+ ent.label_].append(ent.text.strip())

    return out_dict 


out_dict = text_cleaner(accounts_txt)

#print(out_dict)

#for t in out_dict['tokens']:
#    print(t)


In [53]:
client = py.MongoClient(MONGO_URI)
db = client.ch

for f in db.docs.find({'filename' : '00363381_accounts_2015-05-31.xhtml'}):
    db.docs.update_one({'_id': f['_id']},
                   {'$set': out_dict })


{'_id': ObjectId('5acdba86d4c21fc0812bf81e'), 'submit_date': '2015-05-31', 'doctype': 'accounts', 'company_number': '00363381', 'filetype': 'xhtml', 'name': '00363381_accounts_2015-05-31', 'azure_folder': 'xhtml', 'container': 'companieshouseapi', 'filename': '00363381_accounts_2015-05-31.xhtml', 'blob_file': 'xhtml/00363381_accounts_2015-05-31.xhtml', 'tokens': ['registered', 'number', 'england', 'wales', 'abbreviated', 'unaudited', 'accounts', 'year', 'ended', '31st', 'may', 'whitehouse', '&', 'hartley', 'limited', 'whitehouse', '&', 'hartley', 'limited', 'registered', 'number', 'contents', 'abbreviated', 'accounts', 'year', 'end', '31st', 'may', 'page', 'company', 'information', 'abbreviated', 'balance', 'sheet', 'notes', 'abbreviated', 'accounts', 'whitehouse', '&', 'hartley', 'limited', 'company', 'information', 'year', 'end', '31st', 'may', 'directors', 'a', 'j', 'power', 'mrs', 'p', 'p', 'power', 'secretary', 'a', 'j', 'power', 'registered', 'office', 'arbor', 'house', 'broadway

In [54]:
db = client.ch
print(db.docs.find_one({'filename' : '00363381_accounts_2015-05-31.xhtml'}))

{'_id': ObjectId('5acdba86d4c21fc0812bf81e'), 'submit_date': '2015-05-31', 'doctype': 'accounts', 'company_number': '00363381', 'filetype': 'xhtml', 'name': '00363381_accounts_2015-05-31', 'azure_folder': 'xhtml', 'container': 'companieshouseapi', 'filename': '00363381_accounts_2015-05-31.xhtml', 'blob_file': 'xhtml/00363381_accounts_2015-05-31.xhtml', 'tokens': ['registered', 'number', 'england', 'wales', 'abbreviated', 'unaudited', 'accounts', 'year', 'ended', '31st', 'may', 'whitehouse', '&', 'hartley', 'limited', 'whitehouse', '&', 'hartley', 'limited', 'registered', 'number', 'contents', 'abbreviated', 'accounts', 'year', 'end', '31st', 'may', 'page', 'company', 'information', 'abbreviated', 'balance', 'sheet', 'notes', 'abbreviated', 'accounts', 'whitehouse', '&', 'hartley', 'limited', 'company', 'information', 'year', 'end', '31st', 'may', 'directors', 'a', 'j', 'power', 'mrs', 'p', 'p', 'power', 'secretary', 'a', 'j', 'power', 'registered', 'office', 'arbor', 'house', 'broadway

In [ ]:
def main():
    """Requires the mongodb connection string as an arguement"""
    MONGO_URI = sys.argv[1]
    
    spacy.load('en')
    client = py.MongoClient(MONGO_URI)
    db = client.ch
    out_dict = text_cleaner(accounts_txt)
    
    for f in db.docs.find({'filename' : '00363381_accounts_2015-05-31.xhtml'}):
        db.docs.update_one({'_id': f['_id']},
                           {'$set': out_dict })

In [66]:
import re
postcode_regex = "([Gg][Ii][Rr] 0[Aa]{2})|((([A-Za-z][0-9]{1,2})|(([A-Za-z][A-Ha-hJ-Yj-y][0-9]{1,2})|(([A-Za-z][0-9][A-Za-z])|([A-Za-z][A-Ha-hJ-Yj-y][0-9]?[A-Za-z]))))\s?[0-9][A-Za-z]{2})"
p = re.compile(postcode_regex)

for f in p.finditer(accounts_txt):
    print(f.group(0))

pcodes = [f.group(0) for f in p.finditer(accounts_txt)]
print(set(pcodes))

WS1 2AN
WS1 2AN
{'WS1 2AN'}


In [19]:
import requests
import json
import re
from geojson import Point, Feature, FeatureCollection


def geocoder (pcodes):
    """Geocodes a list of postcodes to produce a MultiPoint geojson object"""
    
    headers = {"Content-Type": "application/json"}
    url = "https://api.postcodes.io/postcodes?filter=postcode,longitude,latitude"

    pc_data = { 'postcodes' : list(set(pcodes)) }

    response = requests.post(url, data=json.dumps(pc_data), headers=headers)
    resp_obj = json.loads(response.text)


    #for a featurecollection - would be ideal but not sure it works well with mongo
    """fc = []

    for r in [ ro['result'] for ro in resp_obj['result']]:
        p = Point((r['longitude'], r['latitude']))
        f = Feature(geometry=p, properties={"postcode": r['postcode']})
        fc.append(f)

    features = FeatureCollection(fc)
    """

    #for a multipoint
    ps = [ (ro['result']['longitude'],ro['result']['latitude']) for ro in resp_obj['result']]

    mp = MultiPoint(ps)

    #print(mp)
    
    return mp

postcode_regex = "([Gg][Ii][Rr] 0[Aa]{2})|((([A-Za-z][0-9]{1,2})|(([A-Za-z][A-Ha-hJ-Yj-y][0-9]{1,2})|(([A-Za-z][0-9][A-Za-z])|([A-Za-z][A-Ha-hJ-Yj-y][0-9]?[A-Za-z]))))\s?[0-9][A-Za-z]{2})"
p = re.compile(postcode_regex)

for f in p.finditer(accounts_txt):
    print(f.group(0))

pcodes = [f.group(0) for f in p.finditer(accounts_txt)]

mp = geocoder(pcodes)


WS1 2AN
WS1 2AN
{"coordinates": [[-1.97400818160824, 52.5884254864852]], "type": "MultiPoint"}


In [3]:
print([ ro['result'] for ro in resp_obj['result']])

[{'postcode': 'PR3 0SG', 'longitude': -2.74625456434894, 'latitude': 53.8566242029322}, {'postcode': 'EX16 5BL', 'longitude': -3.50192644641646, 'latitude': 50.9000683960022}, {'postcode': 'M45 6GN', 'longitude': -2.29705263978866, 'latitude': 53.550284089076}]


In [12]:
from geojson import Point
Point(-2.746,53.856)

TypeError: 'float' object is not iterable